In [1]:
!pip install --upgrade google-cloud-bigquery --qq


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: --qq


In [5]:
from google.cloud import bigquery
from google.colab import drive
drive.mount("/content/drive/")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
GOOGLE_AUTH_KEY_PATH = "/content/drive/My Drive/BigQueryCredentials/My_First_Project-aa369763a577.json"

from google.oauth2 import service_account

credentials = service_account.Credentials.from_service_account_file(GOOGLE_AUTH_KEY_PATH)
project_id = "bq-chicago-crime"
client = bigquery.Client(credentials=credentials, project=project_id)

In [0]:
dataset_ref = client.dataset("chicago_crime", project="bigquery-public-data")
dataset = client.get_dataset(dataset_ref)

In [14]:
tables = list(client.list_tables(dataset))
for i in tables:
    print(i.table_id)

crime


In [15]:
table_ref = dataset_ref.table("crime")
table = client.get_table(table_ref)
table.schema

[SchemaField('unique_key', 'INTEGER', 'REQUIRED', 'Unique identifier for the record.', ()),
 SchemaField('case_number', 'STRING', 'NULLABLE', 'The Chicago Police Department RD Number (Records Division Number), which is unique to the incident.', ()),
 SchemaField('date', 'TIMESTAMP', 'NULLABLE', 'Date when the incident occurred. this is sometimes a best estimate.', ()),
 SchemaField('block', 'STRING', 'NULLABLE', 'The partially redacted address where the incident occurred, placing it on the same block as the actual address.', ()),
 SchemaField('iucr', 'STRING', 'NULLABLE', 'The Illinois Unifrom Crime Reporting code. This is directly linked to the Primary Type and Description. See the list of IUCR codes at https://data.cityofchicago.org/d/c7ck-438e.', ()),
 SchemaField('primary_type', 'STRING', 'NULLABLE', 'The primary description of the IUCR code.', ()),
 SchemaField('description', 'STRING', 'NULLABLE', 'The secondary description of the IUCR code, a subcategory of the primary descriptio

In [16]:
client.list_rows(table, max_results=20).to_dataframe()

,unique_key,case_number,date,block,iucr,primary_type,description,location_description,arrest,domestic,beat,district,ward,community_area,fbi_code,x_coordinate,y_coordinate,year,updated_on,latitude,longitude,location
0,11246651,JB173154,2017-06-15 10:00:00+00:00,052XX W SEMINOLE ST,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,RESIDENCE,False,False,1621,16,45,11,11,NaN,NaN,2017,2018-03-05 15:57:45+00:00,NaN,NaN,None
1,2464669,HH793944,2002-10-11 17:53:00+00:00,054XX W 63RD ST,1206,DECEPTIVE PRACTICE,"THEFT BY LESSEE,MOTOR VEH",OTHER,True,False,813,8,13,64,11,NaN,NaN,2002,2015-08-17 15:03:40+00:00,NaN,NaN,None
2,9756103,HX405792,2009-10-01 00:01:00+00:00,003XX W ROOT ST,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,RESIDENCE,False,False,925,9,3,37,11,NaN,NaN,2009,2015-08-17 15:03:40+00:00,NaN,NaN,None
3,8144810,HT376894,2008-07-04 00:00:00+00:00,091XX S DOBSON AVE,1195,DECEPTIVE PRACTICE,FINAN EXPLOIT-ELDERLY/DISABLED,RESIDENCE,False,True,413,4,8,47,11,NaN,NaN,2008,2015-08-17 15:03:40+00:00,NaN,NaN,None
4,8190135,HT423218,2008-05-23 12:00:00+00:00,039XX S LAKE PARK AVE,1130,DECEPTIVE PRACTICE,FRAUD OR CONFIDENCE GAME,VACANT LOT/LAND,False,False,2122,2,4,36,11,NaN,NaN,2008,2015-08-17 15:03:40+00:00,NaN,NaN,None
5,10752605,HZ514885,2007-09-01 21:00:00+00:00,052XX N MILWAUKEE AVE,1563,SEX OFFENSE,CRIMINAL SEXUAL ABUSE,RESIDENCE,False,False,1623,16,45,11,17,NaN,NaN,2007,2016-11-19 15:49:42+00:00,NaN,NaN,None
6,9879513,HX511292,2007-11-15 00:01:00+00:00,097XX S EWING AVE,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,RESIDENCE,False,False,432,4,10,52,11,NaN,NaN,2007,2015-08-31 15:43:09+00:00,NaN,NaN,None
7,2313219,HH605353,2002-08-21 00:00:00+00:00,037XX S ASHLAND AVE,1206,DECEPTIVE PRACTICE,"THEFT BY LESSEE,MOTOR VEH",OTHER,False,False,922,9,11,59,11,NaN,NaN,2002,2015-08-17 15:03:40+00:00,NaN,NaN,None
8,2543929,HJ123587,2003-01-13 13:35:00+00:00,131XX S EBERHART AVE,0630,BURGLARY,ATTEMPT FORCIBLE ENTRY,RESIDENCE,True,False,533,5,9,54,05,NaN,NaN,2003,2015-08-17 15:03:40+00:00,NaN,NaN,None
9,10825882,JA127213,2015-10-23 00:00:00+00:00,036XX S HONORE ST,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,None,False,False,912,9,11,59,11,NaN,NaN,2015,2017-01-24 15:55:11+00:00,NaN,NaN,None


![structure of big query client environment](https://i.imgur.com/biYqbUB.png)

Working with big datasets
BigQuery datasets can be huge. We allow you to do a lot of computation for free, but everyone has some limit.

Each Kaggle user can scan 5TB every 30 days for free. Once you hit that limit, you'll have to wait for it to reset.

The biggest dataset currently on Kaggle is 3TB, so you can go through your 30-day limit in a couple queries if you aren't careful.

Don't worry though: we'll teach you how to avoid scanning too much data at once, so that you don't run over your limit.

To begin,you can estimate the size of any query before running it. Here is an example using the (very large!) Hacker News dataset. To see how much data a query will scan, we create a QueryJobConfig object and set the dry_run parameter to True.

In [0]:
# Query to get the score column from every row where the type column has value "job"
query = """
        SELECT latitude, longitude	
        FROM `bigquery-public-data.chicago_crime.crime`
        WHERE arrest = 'True'
        """

# Create a QueryJobConfig object to estimate size of query without running it
dry_run_config = bigquery.QueryJobConfig(dry_run=True)

# API request - dry run query to estimate costs
dry_run_query_job = client.query(query, job_config=dry_run_config)

print("This query will process {} bytes.".format(dry_run_query_job.total_bytes_processed))

You can also specify a parameter when running the query to limit how much data you are willing to scan. Here's an example with a low limit.

In [0]:
# Only run the query if it's less than 1 MB
ONE_MB = 1000*1000
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=ONE_MB)

# Set up the query (will only run if it's less than 1 MB)
safe_query_job = client.query(query, job_config=safe_config)

# API request - try to run the query, and return a pandas DataFrame
safe_query_job.to_dataframe()

```

score	title
    0	6.0	Interns! - Mixpanel Analytics [SF]
    1	2.0	Senior Software Engineer at Mertado (Palo Alto...
    2	15.0	Heyzap's looking for startup Graphic/UI/UX Des...
    3	4.0	Airbnb - Frontend Engineer
    4	65.0	Rails developer needed for stealth/VC funded Y...
...	...	...
12302	3.0	BackType is hiring a BD engineer
12303	3.0	Software Engineers Wanted (bounty: iPad for ea...
12304	3.0	UI / UX designer - Mixpanel
12305	3.0	Mixpanel is hiring a great frontend developer
12306	3.0	BackType is hiring front and back-end developers
```



In this case, the query was cancelled, because the limit of 1 MB was exceeded. However, we can increase the limit to run the query successfully!

In [0]:
# Only run the query if it's less than 1 GB
ONE_GB = 1000*1000*1000
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=ONE_GB)

# Set up the query (will only run if it's less than 1 GB)
safe_query_job = client.query(query, job_config=safe_config)

# API request - try to run the query, and return a pandas DataFrame
job_post_scores = safe_query_job.to_dataframe()

# Print average score for job posts
job_post_scores.score.mean()

1.942954448701575

In [0]:
# API request - run the query, and return a pandas DataFrame
long_lat_arrests_true = query_job.to_dataframe()

In [0]:
# What five cities have the most measurements?
long_lat_arrests_true.city.value_counts().head()